# Gene–Disease Network Enrichment with UniProt & BioPortal


## Table of Contents
1. [Environment Setup](#setup)
2. [Imports](#imports)
3. [Configuration & Parameters](#config)
4. [Load Graph](#load-graph)
5. [Enrich Genes with UniProt](#enrich-genes)
6. [Enrich Diseases with MeSH/OMIM](#enrich-diseases)
7. [Visualization](#visualization)
8. [Save & Export](#save-export)
9. [Next Steps](#next-steps)


## 1. Environment Setup <a id='setup'></a>
Install required libraries

In [20]:
!pip install psycopg2-binary pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.2 MB/s eta 0:00:00


In [28]:
!pip install gql[requests]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.8 MB/s eta 0:00:00


In [32]:
!pip install mygene


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.4 MB/s eta 0:00:00


In [37]:
# --- Setup ---
!pip install -q mygene gql[requests] tqdm

import mygene
from tqdm import tqdm
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# --- Connect to Open Targets GraphQL ---
transport = RequestsHTTPTransport(
    url="https://api.platform.opentargets.org/api/v4/graphql",
    verify=True,
    retries=3,
)
client = Client(transport=transport, fetch_schema_from_transport=True)

# --- Step 1: Map Entrez Gene IDs to Ensembl IDs and HGNC Symbols ---
print("🔎 Mapping Gene::#### nodes to Ensembl IDs...")

mg = mygene.MyGeneInfo()
gene_ids = [n.split("::")[1] for n, d in G.nodes(data=True)
            if d.get("label") == "Gene" and "::" in n]

entrez_to_ensembl = {}
for i in tqdm(range(0, len(gene_ids), 1000)):
    chunk = gene_ids[i:i+1000]
    results = mg.querymany(chunk, scopes='entrezgene', fields='symbol,ensembl.gene', species='human')
    for r in results:
        if "ensembl" in r and "symbol" in r:
            ens = r["ensembl"]
            if isinstance(ens, dict):
                entrez_to_ensembl[r["query"]] = {
                    "symbol": r["symbol"],
                    "ensembl_id": ens["gene"]
                }

# Apply mappings to your graph
for node in G.nodes:
    if node.startswith("Gene::"):
        entrez = node.split("::")[1]
        info = entrez_to_ensembl.get(entrez)
        if info:
            G.nodes[node]["symbol"] = info["symbol"]
            G.nodes[node]["ensembl_id"] = info["ensembl_id"]

print(f"✅ Mapped {len(entrez_to_ensembl)} genes with Ensembl IDs.")

# --- Step 2: Enrich with Open Targets Associations ---
print("🔗 Enriching gene–disease links from Open Targets...")

added_diseases = 0
added_edges = 0
skipped = 0

# Snapshot to avoid iteration issues
gene_nodes = [(n, d.get("ensembl_id")) for n, d in G.nodes(data=True)
              if d.get("label") == "Gene" and d.get("ensembl_id")]

for node, ensembl_id in tqdm(gene_nodes[:500]):  # You can raise the limit as needed
    query = gql(f"""
    query {{
      target(ensemblId: "{ensembl_id}") {{
        associatedDiseases(page: {{ index: 0, size: 10 }}) {{
          rows {{
            disease {{ id name }}
            score
          }}
        }}
      }}
    }}
    """)

    try:
        result = client.execute(query)
        target_block = result.get("target")
        if not target_block:
            skipped += 1
            continue

        rows = target_block.get("associatedDiseases", {}).get("rows", [])
        for row in rows:
            raw_id = row["disease"]["id"]
            disease_id = f"Disease::{raw_id}"
            disease_name = row["disease"]["name"]
            score = row.get("score", 0)

            # Always assign disease label and name
            if not G.has_node(disease_id):
                G.add_node(disease_id, label="Disease", name=disease_name)
                added_diseases += 1
            else:
                G.nodes[disease_id]["label"] = "Disease"
                G.nodes[disease_id]["name"] = disease_name

            if not G.has_edge(node, disease_id):
                G.add_edge(node, disease_id, relation="associated_with", score=score)
                added_edges += 1

    except Exception as e:
        print(f"⚠️ Error for {ensembl_id}: {e}")
        skipped += 1

print(f"\n✅ Enrichment complete.")
print(f"📦 Added {added_diseases} disease nodes and 🔗 {added_edges} edges.")
print(f"⏭️ Skipped {skipped} queries.")


🔎 Mapping Gene::#### nodes to Ensembl IDs...


  0%|          | 0/21 [00:00<?, ?it/s]WARNING:biothings.client:Input sequence provided is already in string format. No operation performed
INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
  5%|▍         | 1/21 [00:01<00:39,  1.98s/it]WARNING:biothings.client:Input sequence provided is already in string format. No operation performed
INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
 10%|▉         | 2/21 [00:03<00:35,  1.87s/it]WARNING:biothings.client:Input sequence provided is already in string format. No operation performed
INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
 14%|█▍        | 3/21 [00:0

✅ Mapped 17910 genes with Ensembl IDs.
🔗 Enriching gene–disease links from Open Targets...


100%|██████████| 500/500 [01:53<00:00,  4.41it/s]


✅ Enrichment complete.
📦 Added 1197 disease nodes and 🔗 3807 edges.
⏭️ Skipped 3 queries.


In [38]:
!pip install -q fuzzywuzzy python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.4 MB/s eta 0:00:00


## 2. Imports <a id='imports'></a>

In [3]:
import pandas as pd
import networkx as nx

# Load your downloaded files
nodes_df = pd.read_csv("hetionet-nodes.tsv", sep='\t')
edges_df = pd.read_csv("hetionet-edges.sif.gz", sep='\t')

print(f"✅ Nodes loaded: {len(nodes_df)}")
print(f"✅ Edges loaded: {len(edges_df)}")

# Build the graph
G = nx.MultiDiGraph()

# Add nodes
for _, row in nodes_df.iterrows():
    G.add_node(row["id"], label=row["kind"], name=row["name"])

# Add edges
for _, row in edges_df.iterrows():
    G.add_edge(row["source"], row["target"], relation=row["metaedge"])

print(f"✅ Final graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")


✅ Nodes loaded: 47031
✅ Edges loaded: 2250197
✅ Final graph: 47031 nodes, 2250197 edges


In [4]:
import requests
import time

def enrich_genes_with_uniprot(G, max_genes=200):
    print("🧬 Enriching genes from UniProt...")

    # Filter gene nodes
    gene_nodes = [n for n, d in G.nodes(data=True) if d.get("label") == "Gene"]
    gene_nodes = gene_nodes[:max_genes]  # limit for speed

    for node_id in gene_nodes:
        gene_name = G.nodes[node_id].get("name")
        if not gene_name:
            continue

        url = f"https://rest.uniprot.org/uniprotkb/search?query=gene_exact:{gene_name}&format=json&size=1"
        try:
            response = requests.get(url)
            data = response.json()

            if data.get("results"):
                record = data["results"][0]
                G.nodes[node_id]["uniprot_id"] = record.get("primaryAccession", "")
                G.nodes[node_id]["uniprot_description"] = record.get("proteinDescription", {}).get("recommendedName", {}).get("fullName", {}).get("value", "")
                synonyms = record.get("gene", [{}])[0].get("synonyms", [])
                G.nodes[node_id]["gene_synonyms"] = [s.get("value") for s in synonyms]

        except Exception as e:
            print(f"⚠️ {gene_name}: {e}")

        time.sleep(0.1)  # avoid rate limit

    print("✅ Gene enrichment complete.")


In [15]:
import requests
import time
from tqdm import tqdm

# MONDO Normalizer endpoint
normalizer_url = "https://name-resolution-normalizer.prod.monarchinitiative.org/resolve"

# Get first 200 disease nodes
disease_nodes = [n for n, d in G.nodes(data=True) if d.get("label") == "Disease"]
disease_nodes = disease_nodes[:200]

added_nodes = 0
added_edges = 0
linked_pairs = []

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Loop through each disease node and resolve to MONDO
for node_id in tqdm(disease_nodes, desc="🌐 MONDO Normalizer Matching"):
    disease_name = G.nodes[node_id].get("name", "")
    if not disease_name:
        continue

    try:
        payload = {"curie": "", "input": disease_name}
        response = requests.post(normalizer_url, json=payload, headers=headers, timeout=6)
        if response.status_code == 200:
            results = response.json().get("normalized_info", [])
            if results:
                best = results[0]
                mondo_curie = best["id"]["identifier"]  # e.g. MONDO:0005294
                mondo_id = f"MONDO::{mondo_curie.split(':')[-1]}"
                mondo_label = best.get("label", "")

                if not G.has_node(mondo_id):
                    G.add_node(mondo_id,
                               label="Ontology",
                               name=mondo_label,
                               source="MONDO",
                               original_uri=mondo_curie)
                    added_nodes += 1

                if not G.has_edge(node_id, mondo_id):
                    G.add_edge(node_id, mondo_id, relation="has_ontology_term")
                    added_edges += 1
                    linked_pairs.append((G.nodes[node_id]["name"], mondo_label, mondo_id))

    except Exception as e:
        print(f"⚠️ {disease_name}: {e}")
    time.sleep(0.25)

# Summary
print(f"\n✅ MONDO Normalizer enrichment complete.")
print(f"📦 Added {added_nodes} MONDO nodes and 🔗 {added_edges} edges.\n")

for disease, mondo_label, mondo_node in linked_pairs[:10]:
    print(f"🔗 {disease} → {mondo_label} → {mondo_node}")


🌐 MONDO Normalizer Matching:   0%|          | 0/137 [00:00<?, ?it/s]

⚠️ idiopathic pulmonary fibrosis: HTTPSConnectionPool(host='name-resolution-normalizer.prod.monarchinitiative.org', port=443): Max retries exceeded with url: /resolve (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7dfdd669a950>: Failed to resolve 'name-resolution-normalizer.prod.monarchinitiative.org' ([Errno -2] Name or service not known)"))


🌐 MONDO Normalizer Matching:   1%|          | 1/137 [00:00<00:40,  3.37it/s]

⚠️ restless legs syndrome: HTTPSConnectionPool(host='name-resolution-normalizer.prod.monarchinitiative.org', port=443): Max retries exceeded with url: /resolve (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7dfdd66a8bd0>: Failed to resolve 'name-resolution-normalizer.prod.monarchinitiative.org' ([Errno -2] Name or service not known)"))


🌐 MONDO Normalizer Matching:   1%|▏         | 2/137 [00:00<00:39,  3.43it/s]

...

⚠️ pleural cancer: HTTPSConnectionPool(host='name-resolution-normalizer.prod.monarchinitiative.org', port=443): Max retries exceeded with url: /resolve (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7dfdd66aba90>: Failed to resolve 'name-resolution-normalizer.prod.monarchinitiative.org' ([Errno -2] Name or service not known)"))


🌐 MONDO Normalizer Matching:  99%|█████████▉| 136/137 [00:35<00:00,  3.82it/s]

⚠️ obesity: HTTPSConnectionPool(host='name-resolution-normalizer.prod.monarchinitiative.org', port=443): Max retries exceeded with url: /resolve (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7dfdd66c0990>: Failed to resolve 'name-resolution-normalizer.prod.monarchinitiative.org' ([Errno -2] Name or service not known)"))


🌐 MONDO Normalizer Matching: 100%|██████████| 137/137 [00:35<00:00,  3.81it/s]


✅ MONDO Normalizer enrichment complete.
📦 Added 0 MONDO nodes and 🔗 0 edges.



In [19]:
import requests
import time
from tqdm import tqdm

# Limit to 100 diseases for enrichment
disease_nodes = [n for n, d in G.nodes(data=True) if d.get("label") == "Disease"]
disease_nodes = disease_nodes[:100]

added_trials = 0
added_trial_edges = 0
linked_trials = []

import urllib.parse

def search_clinical_trials(disease_name):
    """Robust ClinicalTrials.gov search using raw expr string."""
    url = "https://clinicaltrials.gov/api/query/study_fields"

    params = {
        "expr": f'"{disease_name}"',  # wrap in quotes to allow multi-word phrases
        "fields": "NCTId,BriefTitle",
        "min_rnk": 1,
        "max_rnk": 3,
        "fmt": "json"
    }

    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code != 200:
            print(f"❌ HTTP error {response.status_code} for {disease_name}")
            return []

        data = response.json()
        studies = data.get("StudyFieldsResponse", {}).get("StudyFields", [])
        return [(s["NCTId"][0], s["BriefTitle"][0]) for s in studies if s.get("NCTId") and s.get("BriefTitle")]

    except Exception as e:
        print(f"⚠️ Exception for {disease_name}: {e}")
        return []


# Enrich graph
for node_id in tqdm(disease_nodes, desc="🔬 Linking clinical trials"):
    disease_name = G.nodes[node_id].get("name")
    if not disease_name:
        continue

    trials = search_clinical_trials(disease_name)
    for nct_id, title in trials:
        trial_node = f"ClinicalTrial::{nct_id}"

        if not G.has_node(trial_node):
            G.add_node(trial_node, label="ClinicalTrial", name=title)
            added_trials += 1

        if not G.has_edge(node_id, trial_node):
            G.add_edge(node_id, trial_node, relation="studied_in")
            added_trial_edges += 1
            linked_trials.append((disease_name, nct_id, title))

    time.sleep(0.3)

print(f"\n✅ Clinical trial linking complete.")
print(f"📦 Added {added_trials} trial nodes and 🔗 {added_trial_edges} edges.")
for dname, nct, title in linked_trials[:10]:
    print(f"🔗 {dname} → {nct} → {title}")



🔬 Linking clinical trials:   0%|          | 0/100 [00:00<?, ?it/s]

❌ HTTP error 404 for idiopathic pulmonary fibrosis


🔬 Linking clinical trials:   1%|          | 1/100 [00:00<00:37,  2.63it/s]

❌ HTTP error 404 for restless legs syndrome


🔬 Linking clinical trials:   2%|▏         | 2/100 [00:00<00:37,  2.64it/s]

...

❌ HTTP error 404 for skin cancer


🔬 Linking clinical trials:  99%|█████████▉| 99/100 [00:37<00:00,  2.64it/s]

❌ HTTP error 404 for systemic scleroderma


🔬 Linking clinical trials: 100%|██████████| 100/100 [00:38<00:00,  2.62it/s]


✅ Clinical trial linking complete.
📦 Added 0 trial nodes and 🔗 0 edges.


In [21]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="aact-db.ctti-clinicaltrials.org",
    dbname="aact",
    user="theiman",
    password="Pth_082704",
    port=5432
)


In [29]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

transport = RequestsHTTPTransport(
    url="https://api.platform.opentargets.org/api/v4/graphql",
    verify=True,
    retries=3,
)

client = Client(transport=transport, fetch_schema_from_transport=True)

query = gql("""
query GetTargetAssociations {
  target(ensemblId: "ENSG00000155657") {
    approvedSymbol
    associatedDiseases(page: { index: 0, size: 5 }) {
      count
      rows {
        disease {
          id
          name
        }
        score
      }
    }
  }
}
""")

result = client.execute(query)
print(result)


{'target': {'approvedSymbol': 'TTN', 'associatedDiseases': {'count': 828, 'rows': [{'disease': {'id': 'EFO_0000407', 'name': 'dilated cardiomyopathy'}, 'score': 0.8463293516985615}, {'disease': {'id': 'MONDO_0011400', 'name': 'dilated cardiomyopathy 1G'}, 'score': 0.806061786200876}, {'disease': {'id': 'EFO_0000318', 'name': 'cardiomyopathy'}, 'score': 0.8040243189083583}, {'disease': {'id': 'MONDO_0010870', 'name': 'tibial muscular dystrophy'}, 'score': 0.7954923412028356}, {'disease': {'id': 'MONDO_0012714', 'name': 'early-onset myopathy with fatal cardiomyopathy'}, 'score': 0.7838356772448081}]}}}


In [33]:
import mygene

mg = mygene.MyGeneInfo()
gene_ids = [n.split("::")[1] for n, _ in G.nodes(data=True) if n.startswith("Gene::")]

# Query MyGene.info for mappings
results = mg.querymany(gene_ids[:100], scopes='entrezgene', fields='symbol,ensembl.gene', species='human')

# Build mapping
entrez_to_ensembl = {}
for r in results:
    if "ensembl" in r and "symbol" in r:
        ens = r["ensembl"]
        if isinstance(ens, dict):
            entrez_to_ensembl[r["query"]] = {
                "symbol": r["symbol"],
                "ensembl_id": ens["gene"]
            }

print(list(entrez_to_ensembl.items())[:5])


INFO:biothings.client:querying 1-100 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.


[('1', {'symbol': 'A1BG', 'ensembl_id': 'ENSG00000121410'}), ('10', {'symbol': 'NAT2', 'ensembl_id': 'ENSG00000156006'}), ('100', {'symbol': 'ADA', 'ensembl_id': 'ENSG00000196839'}), ('1000', {'symbol': 'CDH2', 'ensembl_id': 'ENSG00000170558'}), ('100008586', {'symbol': 'GAGE12F', 'ensembl_id': 'ENSG00000236362'})]


In [34]:
from gql import gql
from tqdm import tqdm

added_diseases = 0
added_edges = 0

# Update graph with mapping attributes
for node in G.nodes:
    if node.startswith("Gene::"):
        entrez = node.split("::")[1]
        if entrez in entrez_to_ensembl:
            G.nodes[node]["symbol"] = entrez_to_ensembl[entrez]["symbol"]
            G.nodes[node]["ensembl_id"] = entrez_to_ensembl[entrez]["ensembl_id"]

# Enrich via GraphQL
# Take a snapshot to avoid mutation error
gene_nodes = [(node, data) for node, data in G.nodes(data=True) if data.get("label") == "Gene"]

for node, data in tqdm(gene_nodes):
    ensembl_id = data.get("ensembl_id")
    if not ensembl_id:
        continue

    query = gql(f"""
    query {{
      target(ensemblId: "{ensembl_id}") {{
        associatedDiseases(page: {{ index: 0, size: 10 }}) {{
          rows {{
            disease {{ id name }}
            score
          }}
        }}
      }}
    }}
    """)

    try:
        result = client.execute(query)
        rows = result["target"]["associatedDiseases"]["rows"]

        for row in rows:
            disease_id = f"EFO::{row['disease']['id']}"
            disease_name = row["disease"]["name"]
            score = row["score"]

            if not G.has_node(disease_id):
                G.add_node(disease_id, label="Disease", name=disease_name)

            if not G.has_edge(node, disease_id):
                G.add_edge(node, disease_id, relation="associated_with", score=score)

    except Exception as e:
        print(f"⚠️ Error for {ensembl_id}: {e}")


print(f"\n✅ Finished Open Targets enrichment.")
print(f"📦 Added {added_diseases} new disease nodes and 🔗 {added_edges} edges.")


100%|██████████| 20945/20945 [00:08<00:00, 2461.82it/s]


✅ Finished Open Targets enrichment.
📦 Added 0 new disease nodes and 🔗 0 edges.


In [35]:
from tqdm import tqdm
import mygene

mg = mygene.MyGeneInfo()

gene_ids = [n.split("::")[1] for n, d in G.nodes(data=True)
            if d.get("label") == "Gene" and "::" in n]

# Query MyGene.info in chunks
entrez_to_ensembl = {}
for i in tqdm(range(0, len(gene_ids), 1000)):
    chunk = gene_ids[i:i+1000]
    results = mg.querymany(chunk, scopes='entrezgene', fields='symbol,ensembl.gene', species='human')
    for r in results:
        if "ensembl" in r and "symbol" in r:
            ens = r["ensembl"]
            if isinstance(ens, dict):
                entrez_to_ensembl[r["query"]] = {
                    "symbol": r["symbol"],
                    "ensembl_id": ens["gene"]
                }


  0%|          | 0/21 [00:00<?, ?it/s]WARNING:biothings.client:Input sequence provided is already in string format. No operation performed
INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
  5%|▍         | 1/21 [00:01<00:39,  1.96s/it]WARNING:biothings.client:Input sequence provided is already in string format. No operation performed
INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
 10%|▉         | 2/21 [00:03<00:35,  1.86s/it]WARNING:biothings.client:Input sequence provided is already in string format. No operation performed
INFO:biothings.client:querying 1-1000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
 14%|█▍        | 3/21 [00:0

In [ ]:
from difflib import get_close_matches
from tqdm import tqdm

added_trials = 0
added_links = 0
unmatched_conditions = []

# Snapshot of existing disease names for matching
disease_lookup = {
    d.lower(): n for n, d in nx.get_node_attributes(G, "name").items()
    if G.nodes[n].get("label") == "Disease"
}

print("🔬 Linking clinical trials...")

for _, row in tqdm(trials_df.iterrows(), total=len(trials_df)):
    cond = str(row["condition_name"]).lower().strip()
    nct_id = row["nct_id"]
    title = row["brief_title"]

    match = disease_lookup.get(cond)
    if not match:
        # Try fuzzy match (optional)
        guess = get_close_matches(cond, disease_lookup.keys(), n=1, cutoff=0.9)
        if guess:
            match = disease_lookup[guess[0]]
        else:
            unmatched_conditions.append(cond)
            continue

    trial_node = f"ClinicalTrial::{nct_id}"
    if not G.has_node(trial_node):
        G.add_node(trial_node, label="ClinicalTrial", name=title)
        added_trials += 1

    if not G.has_edge(match, trial_node):
        G.add_edge(match, trial_node, relation="studied_in")
        added_links += 1

print(f"\n✅ Trial linking complete.")
print(f"📦 Added {added_trials} trial nodes and 🔗 {added_links} edges.")
print(f"⏭️ Unmatched conditions: {len(unmatched_conditions)}")


🔬 Linking clinical trials...


100%|██████████| 5000/5000 [01:23<00:00, 60.23it/s]


✅ Trial linking complete.
📦 Added 1260 trial nodes and 🔗 1332 edges.
⏭️ Unmatched conditions: 3661


In [39]:
from fuzzywuzzy import fuzz
from tqdm import tqdm

added_trials = 0
added_links = 0
unmatched_conditions = []

# Snapshot of lowercased disease name → node_id
disease_lookup = {
    d.lower(): n for n, d in nx.get_node_attributes(G, "name").items()
    if G.nodes[n].get("label") == "Disease"
}

print("🔬 Linking clinical trials using fuzzy matching...")

for _, row in tqdm(trials_df.iterrows(), total=len(trials_df)):
    cond = str(row["condition_name"]).lower().strip()
    nct_id = row["nct_id"]
    title = row["brief_title"]

    # Try exact match first
    match_node = disease_lookup.get(cond)

    if not match_node:
        # Fuzzy match if exact fails
        best_match = None
        best_score = 0
        for name in disease_lookup:
            score = fuzz.partial_ratio(cond, name)
            if score > best_score and score >= 85:
                best_score = score
                best_match = name

        if best_match:
            match_node = disease_lookup[best_match]
        else:
            unmatched_conditions.append(cond)
            continue

    trial_node = f"ClinicalTrial::{nct_id}"
    if not G.has_node(trial_node):
        G.add_node(trial_node, label="ClinicalTrial", name=title)
        added_trials += 1

    if not G.has_edge(match_node, trial_node):
        G.add_edge(match_node, trial_node, relation="studied_in")
        added_links += 1

print(f"\n✅ Fuzzy trial linking complete.")
print(f"📦 Added {added_trials} trial nodes and 🔗 {added_links} edges.")
print(f"⏭️ Unmatched conditions: {len(unmatched_conditions)}")


🔬 Linking clinical trials using fuzzy matching...


100%|██████████| 5000/5000 [01:55<00:00, 43.35it/s]


✅ Fuzzy trial linking complete.
📦 Added 2348 trial nodes and 🔗 2662 edges.
⏭️ Unmatched conditions: 2194


In [41]:
import requests
import pandas as pd
from tqdm import tqdm
import time

def fetch_mesh_synonyms(apikey, pages=50):
    base_url = "https://data.bioontology.org/ontologies/MESH/classes"
    params = {
        "apikey": apikey,
        "pagesize": 100
    }

    all_data = []

    for page in tqdm(range(1, pages + 1), desc="🔍 Fetching MeSH terms"):
        params["page"] = page
        try:
            r = requests.get(base_url, params=params)
            r.raise_for_status()
            for entry in r.json().get("collection", []):
                name = entry.get("prefLabel")
                synonyms = entry.get("synonym", [])
                if name:
                    all_data.append({
                        "Preferred_Name": name,
                        "Synonyms": synonyms
                    })
            time.sleep(0.3)
        except Exception as e:
            print(f"⚠️ Page {page} failed: {e}")
            continue

    return pd.DataFrame(all_data)

# Fetch 5000+ terms (~50 pages)
mesh_df = fetch_mesh_synonyms(BIOPORTAL_APIKEY, pages=50)


🔍 Fetching MeSH terms: 100%|██████████| 50/50 [00:45<00:00,  1.11it/s]


## 3. Configuration & Parameters <a id='config'></a>'

In [5]:
enrich_genes_with_uniprot(G, max_genes=200)

🧬 Enriching genes from UniProt...
✅ Gene enrichment complete.


In [7]:
enrich_diseases_with_mesh_omim(G, max_diseases=200)


📚 Enriching diseases with MeSH/OMIM using BioPortal (fuzzy match)...
✅ Fuzzy disease enrichment complete.


In [8]:
# Display enriched diseases
count = 0
for node, data in G.nodes(data=True):
    if data.get("label") == "Disease" and data.get("bioportal_id"):
        print(f"🦠 {data['name']}")
        print(f"   BioPortal ID: {data.get('bioportal_id')}")
        print(f"   Label: {data.get('bioportal_label')}")
        print(f"   Synonyms: {data.get('bioportal_synonyms')}")
        print(f"   Link: {data.get('bioportal_links')}\n")
        count += 1
        if count >= 5:
            break



In [9]:
def enrich_diseases_with_mesh_omim(G, max_diseases=200, apikey="YOUR_API_KEY_HERE"):
    print("📚 Enriching diseases with MeSH/OMIM using BioPortal (with API key)...")

    search_url = "https://data.bioontology.org/search"
    disease_nodes = [n for n, d in G.nodes(data=True) if d.get("label") == "Disease"]
    disease_nodes = disease_nodes[:max_diseases]

    for node_id in disease_nodes:
        disease_name = G.nodes[node_id].get("name")
        if not disease_name:
            continue

        try:
            print(f"🔍 Searching: {disease_name}")
            params = {
                "q": disease_name,
                "ontologies": "MESH,OMIM",
                "format": "json",
                "pagesize": 1,
                "apikey": apikey
            }
            response = requests.get(search_url, params=params)
            data = response.json()

            if data.get("collection"):
                match = data["collection"][0]
                G.nodes[node_id]["bioportal_id"] = match.get("@id")
                G.nodes[node_id]["bioportal_label"] = match.get("prefLabel")
                G.nodes[node_id]["bioportal_synonyms"] = match.get("synonym")
                G.nodes[node_id]["bioportal_links"] = match.get("links", {}).get("self")
                print(f"✅ Match: {match.get('prefLabel')} ({match.get('@id')})")
            else:
                print("❌ No match found")

        except Exception as e:
            print(f"⚠️ Error for {disease_name}: {e}")

        time.sleep(0.3)

    print("✅ Enrichment complete.")


In [10]:
enrich_diseases_with_mesh_omim(G, max_diseases=200, apikey="41a17e7d-a975-4fc3-a0c7-f6a1915ea89a")


📚 Enriching diseases with MeSH/OMIM using BioPortal (with API key)...
🔍 Searching: idiopathic pulmonary fibrosis
✅ Match: Idiopathic Pulmonary Fibrosis (http://purl.bioontology.org/ontology/MESH/D054990)
🔍 Searching: restless legs syndrome
✅ Match: Restless Legs Syndrome (http://purl.bioontology.org/ontology/MESH/D012148)
🔍 Searching: alcohol dependence
✅ Match: ALCOHOL DEPENDENCE (http://purl.bioontology.org/ontology/OMIM/103780)
🔍 Searching: nicotine dependence
✅ Match: Nicotine (http://purl.bioontology.org/ontology/MESH/D009538)
🔍 Searching: lymphatic system cancer
✅ Match: Lymphatic System (http://purl.bioontology.org/ontology/MESH/D008208)
🔍 Searching: pharynx cancer
✅ Match: Pharyngeal Neoplasms (http://purl.bioontology.org/ontology/MESH/D010610)
🔍 Searching: duodenum cancer
✅ Match: Duodenal Neoplasms (http://purl.bioontology.org/ontology/MESH/D004379)
🔍 Searching: ileum cancer
✅ Match: Ileum (http://purl.bioontology.org/ontology/MESH/D007082)
🔍 Searching: leprosy
✅ Match: Lepro

In [11]:
# Show sample enriched diseases
count = 0
for node, data in G.nodes(data=True):
    if data.get("label") == "Disease" and data.get("bioportal_id"):
        print(f"🦠 {data['name']}")
        print(f"   MeSH/OMIM: {data.get('bioportal_label')}")
        print(f"   Link: {data.get('bioportal_links')}\n")
        count += 1
        if count >= 5:
            break


🦠 idiopathic pulmonary fibrosis
   MeSH/OMIM: Idiopathic Pulmonary Fibrosis
   Link: https://data.bioontology.org/ontologies/MESH/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FMESH%2FD054990

🦠 restless legs syndrome
   MeSH/OMIM: Restless Legs Syndrome
   Link: https://data.bioontology.org/ontologies/MESH/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FMESH%2FD012148

🦠 alcohol dependence
   MeSH/OMIM: ALCOHOL DEPENDENCE
   Link: https://data.bioontology.org/ontologies/OMIM/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FOMIM%2F103780

🦠 nicotine dependence
   MeSH/OMIM: Nicotine
   Link: https://data.bioontology.org/ontologies/MESH/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FMESH%2FD009538

🦠 lymphatic system cancer
   MeSH/OMIM: Lymphatic System
   Link: https://data.bioontology.org/ontologies/MESH/classes/http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FMESH%2FD008208



In [12]:
added_nodes = 0
added_edges = 0

for node, data in list(G.nodes(data=True)):
    if data.get("label") == "Disease" and data.get("bioportal_id"):
        mesh_omim_id = data["bioportal_id"]
        label = data.get("bioportal_label", "")
        source = data.get("bioportal_links", "")

        if not G.has_node(mesh_omim_id):
            G.add_node(mesh_omim_id,
                       label="Ontology",
                       name=label,
                       source="BioPortal",
                       original_source=mesh_omim_id)
            added_nodes += 1

        if not G.has_edge(node, mesh_omim_id):
            G.add_edge(node, mesh_omim_id, relation="has_ontology_term")
            added_edges += 1

added_nodes, added_edges


(137, 137)

In [13]:
def enrich_diseases_with_mesh_omim(G, max_diseases=200, apikey="YOUR_API_KEY_HERE"):
    """
    Enrich Disease nodes with MeSH/OMIM terms from BioPortal and structurally link them.
    - Adds new Ontology nodes with simplified IDs (e.g., MeSH::D001943)
    - Adds 'has_ontology_term' edges from Disease → Ontology node
    """
    import requests
    import time

    print("📚 Enriching diseases with MeSH/OMIM using BioPortal...")
    search_url = "https://data.bioontology.org/search"

    disease_nodes = [n for n, d in G.nodes(data=True) if d.get("label") == "Disease"]
    disease_nodes = disease_nodes[:max_diseases]

    added_nodes = 0
    added_edges = 0

    for node_id in disease_nodes:
        disease_name = G.nodes[node_id].get("name")
        if not disease_name:
            continue

        try:
            print(f"🔍 Searching: {disease_name}")
            params = {
                "q": disease_name,
                "ontologies": "MESH,OMIM",
                "format": "json",
                "pagesize": 1,
                "apikey": apikey
            }
            response = requests.get(search_url, params=params)
            data = response.json()

            if data.get("collection"):
                match = data["collection"][0]
                full_uri = match.get("@id")
                label = match.get("prefLabel", "")
                synonyms = match.get("synonym", [])
                source_link = match.get("links", {}).get("self", "")

                # Store on disease node
                G.nodes[node_id]["bioportal_id"] = full_uri
                G.nodes[node_id]["bioportal_label"] = label
                G.nodes[node_id]["bioportal_synonyms"] = synonyms
                G.nodes[node_id]["bioportal_links"] = source_link

                # Create simplified node ID
                if "MESH" in full_uri:
                    readable_id = "MeSH::" + full_uri.split("/")[-1]
                elif "OMIM" in full_uri:
                    readable_id = "OMIM::" + full_uri.split("/")[-1]
                else:
                    readable_id = "Ontology::" + full_uri.split("/")[-1]

                # Add ontology node if missing
                if not G.has_node(readable_id):
                    G.add_node(readable_id,
                               label="Ontology",
                               name=label,
                               source="BioPortal",
                               original_uri=full_uri)
                    added_nodes += 1

                # Add edge
                if not G.has_edge(node_id, readable_id):
                    G.add_edge(node_id, readable_id, relation="has_ontology_term")
                    added_edges += 1

                print(f"✅ Linked: {disease_name} → {readable_id}")

            else:
                print("❌ No match found")

        except Exception as e:
            print(f"⚠️ Error for {disease_name}: {e}")

        time.sleep(0.3)

    print(f"✅ Enrichment complete. Added {added_nodes} nodes and {added_edges} edges.")


In [14]:
enrich_diseases_with_mesh_omim(G, max_diseases=200, apikey="41a17e7d-a975-4fc3-a0c7-f6a1915ea89a")

📚 Enriching diseases with MeSH/OMIM using BioPortal...
🔍 Searching: idiopathic pulmonary fibrosis
✅ Linked: idiopathic pulmonary fibrosis → MeSH::D054990
🔍 Searching: restless legs syndrome
✅ Linked: restless legs syndrome → MeSH::D012148
🔍 Searching: alcohol dependence
✅ Linked: alcohol dependence → OMIM::103780
🔍 Searching: nicotine dependence
✅ Linked: nicotine dependence → MeSH::D009538
🔍 Searching: lymphatic system cancer
✅ Linked: lymphatic system cancer → MeSH::D008208
🔍 Searching: pharynx cancer
✅ Linked: pharynx cancer → MeSH::D010610
🔍 Searching: duodenum cancer
✅ Linked: duodenum cancer → MeSH::D004379
🔍 Searching: ileum cancer
✅ Linked: ileum cancer → MeSH::D007082
🔍 Searching: leprosy
✅ Linked: leprosy → MeSH::D007918
🔍 Searching: prostate cancer
✅ Linked: prostate cancer → OMIM::176807
🔍 Searching: stomach cancer
✅ Linked: stomach cancer → OMIM::MTHU072800
🔍 Searching: celiac disease
✅ Linked: celiac disease → MeSH::D002446
🔍 Searching: Alzheimer's disease
✅ Linked: Alzhe

In [16]:
def enrich_diseases_with_mesh_omim(G, max_diseases=200, apikey="41a17e7d-a975-4fc3-a0c7-f6a1915ea89a"):
    import requests, time

    print("📚 Enriching diseases with MeSH/OMIM using BioPortal (with API key)...")

    search_url = "https://data.bioontology.org/search"
    disease_nodes = [n for n, d in G.nodes(data=True) if d.get("label") == "Disease"]
    disease_nodes = disease_nodes[:max_diseases]

    added = 0
    for node_id in disease_nodes:
        disease_name = G.nodes[node_id].get("name")
        if not disease_name:
            continue

        try:
            print(f"🔍 Searching: {disease_name}")
            params = {
                "q": disease_name,
                "ontologies": "MESH,OMIM",
                "format": "json",
                "pagesize": 1,
                "apikey": apikey
            }
            response = requests.get(search_url, params=params)
            data = response.json()

            if data.get("collection"):
                match = data["collection"][0]
                source = match.get("links", {}).get("ontology", "").split("/")[-1]  # MESH or OMIM
                concept_id = match.get("@id", "").split("/")[-1]
                label = match.get("prefLabel")
                synonyms = match.get("synonym", [])
                uri = match.get("links", {}).get("self")

                if source and concept_id:
                    node_tag = f"{source}::{concept_id}"
                    if not G.has_node(node_tag):
                        G.add_node(node_tag,
                                   label="Ontology",
                                   name=label,
                                   source=source,
                                   synonyms=synonyms,
                                   uri=uri)
                    if not G.has_edge(node_id, node_tag):
                        G.add_edge(node_id, node_tag, relation="has_ontology_term")
                        added += 1

                print(f"✅ Linked: {disease_name} → {label} → {node_tag}")
            else:
                print("❌ No match found")

        except Exception as e:
            print(f"⚠️ Error for {disease_name}: {e}")

        time.sleep(0.3)

    print(f"\n✅ Enrichment complete. Added {added} new ontology links.")


In [17]:
def enrich_diseases_with_mesh_omim(G, max_diseases=200, apikey="41a17e7d-a975-4fc3-a0c7-f6a1915ea89a"):
    import requests, time

    print("📚 Enriching diseases with MeSH/OMIM using BioPortal (with API key)...")

    search_url = "https://data.bioontology.org/search"
    disease_nodes = [n for n, d in G.nodes(data=True) if d.get("label") == "Disease"]
    disease_nodes = disease_nodes[:max_diseases]

    added = 0
    for node_id in disease_nodes:
        disease_name = G.nodes[node_id].get("name")
        if not disease_name:
            continue

        try:
            print(f"🔍 Searching: {disease_name}")
            params = {
                "q": disease_name,
                "ontologies": "MESH,OMIM",
                "format": "json",
                "pagesize": 1,
                "apikey": apikey
            }
            response = requests.get(search_url, params=params)
            data = response.json()

            if data.get("collection"):
                match = data["collection"][0]
                source = match.get("links", {}).get("ontology", "").split("/")[-1]  # MESH or OMIM
                concept_id = match.get("@id", "").split("/")[-1]
                label = match.get("prefLabel")
                synonyms = match.get("synonym", [])
                uri = match.get("links", {}).get("self")

                if source and concept_id:
                    node_tag = f"{source}::{concept_id}"
                    if not G.has_node(node_tag):
                        G.add_node(node_tag,
                                   label="Ontology",
                                   name=label,
                                   source=source,
                                   synonyms=synonyms,
                                   uri=uri)
                    if not G.has_edge(node_id, node_tag):
                        G.add_edge(node_id, node_tag, relation="has_ontology_term")
                        added += 1

                print(f"✅ Linked: {disease_name} → {label} → {node_tag}")
            else:
                print("❌ No match found")

        except Exception as e:
            print(f"⚠️ Error for {disease_name}: {e}")

        time.sleep(0.3)

    print(f"\n✅ Enrichment complete. Added {added} new ontology links.")


In [18]:
enrich_diseases_with_mesh_omim(G, max_diseases=200, apikey="41a17e7d-a975-4fc3-a0c7-f6a1915ea89a")

📚 Enriching diseases with MeSH/OMIM using BioPortal (with API key)...
🔍 Searching: idiopathic pulmonary fibrosis
✅ Linked: idiopathic pulmonary fibrosis → Idiopathic Pulmonary Fibrosis → MESH::D054990
🔍 Searching: restless legs syndrome
✅ Linked: restless legs syndrome → Restless Legs Syndrome → MESH::D012148
🔍 Searching: alcohol dependence
✅ Linked: alcohol dependence → ALCOHOL DEPENDENCE → OMIM::103780
🔍 Searching: nicotine dependence
✅ Linked: nicotine dependence → Nicotine → MESH::D009538
🔍 Searching: lymphatic system cancer
✅ Linked: lymphatic system cancer → Lymphatic System → MESH::D008208
🔍 Searching: pharynx cancer
✅ Linked: pharynx cancer → Pharyngeal Neoplasms → MESH::D010610
🔍 Searching: duodenum cancer
✅ Linked: duodenum cancer → Duodenal Neoplasms → MESH::D004379
🔍 Searching: ileum cancer
✅ Linked: ileum cancer → Ileum → MESH::D007082
🔍 Searching: leprosy
✅ Linked: leprosy → Leprosy → MESH::D007918
🔍 Searching: prostate cancer
✅ Linked: prostate cancer → PROSTATE CANCER →

In [22]:
query = """
SELECT
    c.name AS condition_name,
    s.nct_id,
    s.brief_title
FROM
    conditions c
JOIN
    studies s ON c.nct_id = s.nct_id
WHERE
    s.study_type = 'Interventional'
    AND s.overall_status = 'Recruiting'
    AND c.name IN (
        'skin cancer',
        'obesity',
        'liver cancer',
        'uterine cancer'
    )
LIMIT 50;
"""

trials_df = pd.read_sql(query, conn)
trials_df.head()


/tmp/ipython-input-22-2462142224.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trials_df = pd.read_sql(query, conn)


,condition_name,nct_id,brief_title


In [23]:
query = """
SELECT
    c.name AS condition_name,
    s.nct_id,
    s.brief_title
FROM
    conditions c
JOIN
    studies s ON c.nct_id = s.nct_id
LIMIT 20;
"""

trials_df = pd.read_sql(query, conn)
trials_df.head()


/tmp/ipython-input-23-1134453260.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trials_df = pd.read_sql(query, conn)


,condition_name,nct_id,brief_title
0,Congenital Adrenal Hyperplasia,NCT00000102,Congenital Adrenal Hyperplasia: Calcium Channe...
1,Lead Poisoning,NCT00000104,Does Lead Burden Alter Neuropsychological Deve...
2,Cancer,NCT00000105,Vaccination With Tetanus and KLH to Assess Imm...
3,Rheumatic Diseases,NCT00000106,41.8 Degree Centigrade Whole Body Hyperthermia...
4,"Heart Defects, Congenital",NCT00000107,Body Water Content in Cyanotic Congenital Hear...


In [24]:
query = """
SELECT
    c.name AS condition_name,
    s.nct_id,
    s.brief_title
FROM
    conditions c
JOIN
    studies s ON c.nct_id = s.nct_id
WHERE
    LOWER(c.name) LIKE '%cancer%'
LIMIT 50;
"""

trials_df = pd.read_sql(query, conn)
trials_df.head()

/tmp/ipython-input-24-1860563233.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trials_df = pd.read_sql(query, conn)


,condition_name,nct_id,brief_title
0,Bladder Cancer,NCT06026189,Safely Reduce Cystoscopic Evaluations for Hema...
1,Breast Cancer,NCT04406779,The Frequency of Thyroid Diseases in Women Wit...
2,Early Breast Cancer,NCT05213403,"Innovative ""Scoring System"" in Breast Cancer P..."
3,"Non-colorectal Cancer (Esophagus, Stomach, Liv...",NCT04478175,Prospective Evaluation of a Program for Early ...
4,Head and Neck Cancer,NCT00017277,Radiation Therapy With or Without Epoetin Alfa...


In [25]:
query = """
SELECT
    c.name AS condition_name,
    s.nct_id,
    s.brief_title
FROM
    conditions c
JOIN
    studies s ON c.nct_id = s.nct_id
LIMIT 500;
"""
trials_df = pd.read_sql(query, conn)
trials_df.head()

/tmp/ipython-input-25-3751379518.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trials_df = pd.read_sql(query, conn)


,condition_name,nct_id,brief_title
0,Congenital Adrenal Hyperplasia,NCT00000102,Congenital Adrenal Hyperplasia: Calcium Channe...
1,Lead Poisoning,NCT00000104,Does Lead Burden Alter Neuropsychological Deve...
2,Cancer,NCT00000105,Vaccination With Tetanus and KLH to Assess Imm...
3,Rheumatic Diseases,NCT00000106,41.8 Degree Centigrade Whole Body Hyperthermia...
4,"Heart Defects, Congenital",NCT00000107,Body Water Content in Cyanotic Congenital Hear...


In [26]:
query = """
SELECT DISTINCT
    LOWER(TRIM(c.name)) AS condition_name,
    s.nct_id,
    s.brief_title,
    s.overall_status,
    s.study_type
FROM
    conditions c
JOIN
    studies s ON c.nct_id = s.nct_id
WHERE
    LOWER(s.study_type) = 'interventional'
    AND LOWER(s.overall_status) IN ('recruiting', 'not yet recruiting', 'active, not recruiting')
    AND c.name IS NOT NULL
LIMIT 5000;
"""
trials_df = pd.read_sql(query, conn)
trials_df.head()

/tmp/ipython-input-26-3604274592.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trials_df = pd.read_sql(query, conn)


,condition_name,nct_id,brief_title,overall_status,study_type
0,03 - bacteria,NCT06988514,Stool Collection for Model,RECRUITING,INTERVENTIONAL
1,177lu-eb-fapi,NCT05400967,Diagnosis of Metastatic Tumors on 68Ga-FAPI PE...,RECRUITING,INTERVENTIONAL
2,18f-fdg pet/ct,NCT07048405,Intermittent Cold Exposure and Brown Adipose T...,RECRUITING,INTERVENTIONAL
3,18 years and over,NCT06300827,The Effect of Structured Digital-Based Educati...,RECRUITING,INTERVENTIONAL
4,19 and 20+ b cell hematologic tumors,NCT05388695,To Observe the Dual-target Chimeric Antigen Re...,RECRUITING,INTERVENTIONAL


In [27]:
added_nodes = 0
added_edges = 0
linked = []

for _, row in trials_df.iterrows():
    disease_name = row["condition_name"].strip().lower()
    nct_id = row["nct_id"]
    title = row["brief_title"]

    # Match disease node by name (case-insensitive)
    matched_nodes = [
        n for n, d in G.nodes(data=True)
        if d.get("label") == "Disease" and d.get("name", "").strip().lower() == disease_name
    ]

    for disease_node in matched_nodes:
        trial_node = f"ClinicalTrial::{nct_id}"

        if not G.has_node(trial_node):
            G.add_node(trial_node, label="ClinicalTrial", name=title)
            added_nodes += 1

        if not G.has_edge(disease_node, trial_node):
            G.add_edge(disease_node, trial_node, relation="studied_in")
            added_edges += 1
            linked.append((disease_node, trial_node))

print(f"\n✅ Trial enrichment complete.")
print(f"📦 Added {added_nodes} trial nodes and 🔗 {added_edges} edges.")
print("🧪 Sample links:")
for u, v in linked[:5]:
    print(f"🔗 {u} → studied_in → {v}")



✅ Trial enrichment complete.
📦 Added 63 trial nodes and 🔗 63 edges.
🧪 Sample links:
🔗 Disease::DOID:7693 → studied_in → ClinicalTrial::NCT02548546
🔗 Disease::DOID:7693 → studied_in → ClinicalTrial::NCT03918460
🔗 Disease::DOID:7693 → studied_in → ClinicalTrial::NCT04307992
🔗 Disease::DOID:7693 → studied_in → ClinicalTrial::NCT04500756
🔗 Disease::DOID:7693 → studied_in → ClinicalTrial::NCT04746677


In [30]:
gene_nodes = [
    (n, d.get("name")) for n, d in G.nodes(data=True)
    if d.get("label") == "Gene" and d.get("name")
]


In [36]:
added_diseases = 0
added_edges = 0
skipped = 0

for node, ensembl_id in tqdm(gene_nodes[:100]):  # test range if needed
    query = gql(f"""
    query {{
      target(ensemblId: "{ensembl_id}") {{
        associatedDiseases(page: {{ index: 0, size: 10 }}) {{
          rows {{
            disease {{ id name }}
            score
          }}
        }}
      }}
    }}
    """)
    try:
        result = client.execute(query)
        target_block = result.get("target")
        if not target_block:
            skipped += 1
            continue

        rows = target_block.get("associatedDiseases", {}).get("rows", [])
        for row in rows:
            raw_id = row["disease"]["id"]
            disease_id = f"Disease::{raw_id}"
            disease_name = row["disease"]["name"]
            score = row.get("score", 0)

            # Always set or update name
            if not G.has_node(disease_id):
                G.add_node(disease_id, label="Disease", name=disease_name)
                added_diseases += 1
            else:
                G.nodes[disease_id]["label"] = "Disease"
                G.nodes[disease_id]["name"] = disease_name

            if not G.has_edge(node, disease_id):
                G.add_edge(node, disease_id, relation="associated_with", score=score)
                added_edges += 1

    except Exception as e:
        print(f"⚠️ Error for {ensembl_id}: {e}")
        skipped += 1

print(f"\n✅ Enrichment complete.")
print(f"📦 Added {added_diseases} disease nodes and 🔗 {added_edges} edges.")
print(f"⏭️ Skipped {skipped} queries.")


100%|██████████| 100/100 [00:16<00:00,  5.96it/s]


✅ Enrichment complete.
📦 Added 0 disease nodes and 🔗 0 edges.
⏭️ Skipped 100 queries.


In [40]:
BIOPORTAL_APIKEY = "41a17e7d-a975-4fc3-a0c7-f6a1915ea89a"  # my real API key


## 4. Load or Build Your Graph <a id='load-graph'></a>

In [1]:
pip install pandas networkx matplotlib requests tqdm


## 5. Enrich Genes with UniProt <a id='enrich-genes'></a>

## 6. Enrich Diseases with MeSH/OMIM <a id='enrich-diseases'></a>

In [6]:
def enrich_diseases_with_mesh_omim(G, max_diseases=200):
    print("📚 Enriching diseases with MeSH/OMIM using BioPortal (fuzzy match)...")

    search_url = "https://data.bioontology.org/search"
    disease_nodes = [n for n, d in G.nodes(data=True) if d.get("label") == "Disease"]
    disease_nodes = disease_nodes[:max_diseases]

    for node_id in disease_nodes:
        disease_name = G.nodes[node_id].get("name")
        if not disease_name:
            continue

        try:
            params = {
                "q": disease_name,
                "ontologies": "MESH,OMIM",
                "format": "json",
                "pagesize": 1  # only need top hit
            }
            response = requests.get(search_url, params=params)
            data = response.json()

            if data.get("collection"):
                match = data["collection"][0]
                G.nodes[node_id]["bioportal_id"] = match.get("@id")
                G.nodes[node_id]["bioportal_label"] = match.get("prefLabel")
                G.nodes[node_id]["bioportal_synonyms"] = match.get("synonym")
                G.nodes[node_id]["bioportal_links"] = match.get("links", {}).get("self")

        except Exception as e:
            print(f"⚠️ {disease_name}: {e}")

        time.sleep(0.3)

    print("✅ Fuzzy disease enrichment complete.")


## 7. Results & Visualization <a id='visualization'></a>

## 8. Save & Export <a id='save-export'></a>

## 9. Next Steps & References <a id='next-steps'></a>
- Consider adding Ensembl integration.
- Extend with additional data sources or visualizations.

In [2]:
# Remove corrupted/existing
!rm -f hetionet-nodes.tsv hetionet-edges.sif.gz

# Download both from GitHub directly
!wget https://github.com/hetio/hetionet/raw/master/hetnet/tsv/hetionet-v1.0-nodes.tsv -O hetionet-nodes.tsv
!wget https://github.com/hetio/hetionet/raw/master/hetnet/tsv/hetionet-v1.0-edges.sif.gz -O hetionet-edges.sif.gz


--2025-07-12 20:56:20--  https://github.com/hetio/hetionet/raw/master/hetnet/tsv/hetionet-v1.0-nodes.tsv
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hetio/hetionet/master/hetnet/tsv/hetionet-v1.0-nodes.tsv [following]
--2025-07-12 20:56:20--  https://raw.githubusercontent.com/hetio/hetionet/master/hetnet/tsv/hetionet-v1.0-nodes.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2509984 (2.4M) [text/plain]
Saving to: ‘hetionet-nodes.tsv’

hetionet-nodes.tsv  100%[===================>]   2.39M  --.-KB/s    in 0.05s   

2025-07-12 20:56:20 (45.2 MB/s) - ‘hetionet-nodes.tsv’ saved [2509984/2509

Enriching genes from UniProt

Enriching diseases with MeSH/OMIM using BioPortal

In [ ]:
api key: "41a17e7d-a975-4fc3-a0c7-f6a1915ea89a"

In [ ]:
api key: "41a17e7d-a975-4fc3-a0c7-f6a1915ea89a"


In [31]:
def make_ensembl_query(ensembl_id):
    return gql(f"""
    query {{
      target(ensemblId: "{ensembl_id}") {{
        approvedSymbol
        associatedDiseases(page: {{ index: 0, size: 10 }}) {{
          rows {{
            disease {{ id name }}
            score
          }}
        }}
      }}
    }}
    """)

